<a href="https://colab.research.google.com/github/AdityaMohanty374/ML_Algos/blob/main/GaussianDiscriminantAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('diabetes.csv')

In [ ]:
preg = df['Pregnancies'].tolist()
glu = df['Glucose'].tolist()
bp = df['BloodPressure'].tolist()
skin = df['SkinThickness'].tolist()
ins = df['Insulin'].tolist()
bmi = df['BMI'].tolist()
dpf = df['DiabetesPedigreeFunction'].tolist()
age = df['Age'].tolist()

In [ ]:
outcome = df['Outcome'].tolist()

In [ ]:
#class priors
total_samples = len(outcome)
samples_y1 = 0
for i in range(total_samples):
  if(outcome[i]==1):
    samples_y1+=1
phi = samples_y1/total_samples

In [ ]:
phi

0.3489583333333333

In [ ]:
outcome = np.array(outcome)
np.mean(outcome==1)

np.float64(0.3489583333333333)

In [ ]:
#using np
phi = np.mean(np.array(outcome)==1)

In [ ]:
X = df.drop(columns="Outcome").values  # (m x 8) feature matrix
y = df["Outcome"].values   #(m,)

In [32]:
X.shape

(768, 8)

In [ ]:
y.shape

(768,)

In [31]:
mew0 = np.array([None for _ in range(X.shape[1])])
mew1 = np.array([None for _ in range(X.shape[1])])
mew0.shape[0]

8

In [34]:
for i in range(mew0.shape[0]):
  sum0 = 0
  len0 = 0
  sum1 = 0
  len1 = 0
  for j in range(X.shape[0]):
    if y[j]==0:
      sum0+=X[j][i]
      len0+=1
    else:
      sum1+=X[j][i]
      len1+=1
  mew0[i] = sum0/len0
  mew1[i] = sum1/len1

In [36]:
mew1

array([np.float64(4.865671641791045), np.float64(141.25746268656715),
       np.float64(70.82462686567165), np.float64(22.16417910447761),
       np.float64(100.33582089552239), np.float64(35.14253731343278),
       np.float64(0.5505), np.float64(37.06716417910448)], dtype=object)

In [38]:
#using numpy
mew0 = X[y == 0].mean(axis=0)
mew1 = X[y == 1].mean(axis=0)  #X[y==1] selects only the rows where y == 1
#axis 0 is operation down the rows
#axis 1 is operation across the columns

In [40]:
mew1

array([  4.86567164, 141.25746269,  70.82462687,  22.1641791 ,
       100.3358209 ,  35.14253731,   0.5505    ,  37.06716418])

Step-by-step:
Initialize Sigma to a zero matrix of shape (n × n)

For each sample
𝑥
(
𝑖
)
x
(i)
 :

Get the correct mean: mu = mu0 if y[i] == 0, else mu1

Subtract: diff = x[i] - mu

Compute outer product: diff.T @ diff or np.outer(diff, diff)

Add to Sigma

After the loop, divide Sigma by m

In [45]:
#covariance matrix

sigma = np.zeros((X.shape[1], X.shape[1]))
mu = 0

for i in range(X.shape[0]):
  if y[i] == 0:
    mu = mew0
  else:
    mu = mew1
  diff = X[i]-mu
  sigma+=(np.outer(diff, diff))

sigma = sigma/X.shape[0]

In [46]:
sigma

array([[ 1.07809403e+01,  2.78938627e+00,  8.26207105e+00,
        -5.27477285e+00, -3.97524980e+01, -1.25402924e+00,
        -8.03884974e-02,  1.94493593e+01],
       [ 2.78938627e+00,  7.98665410e+02,  7.55441937e+01,
         1.14353016e+01,  9.95201495e+02,  2.12740976e+01,
         5.94839635e-01,  5.71917522e+01],
       [ 8.26207105e+00,  7.55441937e+01,  3.72575299e+02,
         6.24461308e+01,  1.79196510e+02,  4.00461105e+01,
         1.91843714e-01,  5.09266624e+01],
       [-5.27477285e+00,  1.14353016e+01,  6.24461308e+01,
         2.52721781e+02,  7.84017292e+02,  4.65613723e+01,
         9.02273844e-01, -2.46914521e+01],
       [-3.97524980e+01,  9.95201495e+02,  1.79196510e+02,
         7.84017292e+02,  1.30378335e+04,  1.44867980e+02,
         6.19203039e+00, -9.91865687e+01],
       [-1.25402924e+00,  2.12740976e+01,  4.00461105e+01,
         4.65613723e+01,  1.44867980e+02,  5.67607243e+01,
         2.34179764e-01, -3.10425087e+00],
       [-8.03884974e-02,  5.948396

In [56]:
sigma_det = float(np.linalg.det(sigma)**0.5)
sigma_inv = (np.linalg.inv(sigma))

In [57]:
##prediction class

def multivariate_gaussian(x, mu, sigma_inv, sigma_det):
    n = x.shape[0]
    diff = x - mu
    exponent = -0.5 * np.dot(diff.T, np.dot(sigma_inv, diff))
    coeff = 1.0 / (np.sqrt((2 * np.pi)**n * sigma_det))
    return coeff * np.exp(exponent)


predictions = []

for i in range(X.shape[0]):
  x = X[i]
  px_y0 = multivariate_gaussian(x, mew0, sigma_inv, sigma_det)
  px_y1 = multivariate_gaussian(x, mew1, sigma_inv, sigma_det)
  posterior = (px_y1 * phi) / (px_y1 * phi + px_y0 * (1 - phi))
  predictions.append(1 if posterior >= 0.5 else 0)

In [58]:
predictions

[1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,


In [59]:
accuracy = 0
for i in range(len(predictions)):
  if predictions[i]==y[i]: accuracy+=1

accuracy/len(y)

0.7838541666666666